In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
from actual_stats_calc import get_actual_stats

df = pd.read_csv('tkrb_stats_test.csv')

# UI for individual sword
def create_new_sword(label):
    dd = widgets.Dropdown(options = sorted(df['Name'].unique()), description = f'{label}')
    is_max = widgets.Checkbox(value = True, description = 'Max Stats', disabled = False, indent = False)
    level_is = widgets.IntSlider(value = 1, min = 1, max = 99, description = 'Level:')
    
    def local_update_level_limit(change):
        if 'Kiwame' in dd.value:
            level_is.max = 199
        else:
            level_is.max = 99
    dd.observe(local_update_level_limit, names='value')

    ui_layout = widgets.VBox(
        [widgets.HBox([dd, is_max]), 
        level_is], 
        layout=widgets.Layout(border='1px solid #ddd', padding='10px', margin='5px')
    )
    
    return{
        'dd': dd,
        'is_max': is_max,
        'level_is': level_is,
        'ui': ui_layout,
    }

# sword status calculation
def get_sword_status(name, stype, level):
    TOKU_LEVEL = {
        'Tantou':20,
        'Wakizashi':20,
        'Uchigatana':20, # 2 flower lv20, 3 flower lv25, since Uchi has both then default 20
        'Tachi':25,
        'Ootachi':25,
        'Yari':25,
        'Naginata':25,
        'Tsurugi':25,
    }
    # automatically set status based on level
    if 'Kiwame' in name:
        status = 'Kiwame'
    elif level >= TOKU_LEVEL.get(stype):
        status = 'Toku'
    else:
        status = 'Base'
    return status

# colour code values
def colour_code_stats(vals):
    if not vals: return []
    max_val = max(vals)
    min_val = min(vals)

    results = []
    for val in vals:
        if val == max_val and max_val != min_val:
            results.append(f'<b style = "color:red;">{val} ↑</b>')
        elif val == min_val and max_val != min_val:
            results.append(f'<b style = "color:blue;">{val} ↓</b>')
        else:
            results.append(f'<b>{val}</b>')
    return results

# pre-load set number of swords for upper limit cap on comparison number
SET = 4
sword_group = [create_new_sword(f'Sword {i + 1}') for i in range(SET)]
sword_num_is = widgets.IntSlider(value = 2, min = 1, max = SET, description = 'Number of comparison:')
base_container = widgets.HBox()
output_area = widgets.Output()

def refresh_ui(change = None):
    #update number of shown swords
    sword_count = sword_num_is.value
    base_container.children = [sword_group[i]['ui'] for i in range(sword_count)]


    with output_area:
        clear_output()
        active_groups = sword_group[:sword_count]
        try:
            # get stats for each sword in group
            results = []
            for ag in active_groups:
                name = ag['dd'].value
                level = ag['level_is'].value
                ss = get_sword_status(name, df[df['Name'] == name].iloc[0]['Type'], level)

                # if user checkboxed max
                if ag['is_max'].value and ss != 'Base': ss = f'{ss}_Max'
            
                # get target info and calculate actual battle stats
                character = df[(df['Name'] == name) & (df['Status'] == ss)]
                if character.empty:
                    print(f"Could not find {name}'s {ss} form.")
                else:
                    results.append(get_actual_stats(character.iloc[0], level))

            if len(results) >= 1:
                # setup headers
                stats_names = ['Survival(生存)','Impact(打撃)','Leadership(銃率)','Mobility(機動)','Impulse(衝力)','KillingBlow(必殺)','Scouting(偵察)','Camouflage(隠蔽)']
                # setup table layout
                html = "<table border='1' style='width:100%; border-collapse:collapse; text-align:center;'>"
                # headers
                html += "<tr style='background-color:#f2f2f2;'><th>Stats</th>" + "".join([f"<th>{r['Name']}<br>(Lv.{r.get('Level', '??')})</th>" for r in results]) + "</tr>"
                
                for s in stats_names:
                    raw_vals = [r.get(s, 0) for r in results]
                    colored_vals = colour_code_stats(raw_vals)
                    html += f"<tr><td>{s}</td>" + "".join([f"<td>{cv}</td>" for cv in colored_vals]) + "</tr>"
                
                display(HTML(html + "</table>"))
            
        except Exception as e:
            print(f"D; something happened: {e}")




# observers
sword_num_is.observe(refresh_ui, names = 'value')
for sword in sword_group:
    sword['dd'].observe(refresh_ui, names = 'value')
    sword['level_is'].observe(refresh_ui, names = 'value')
    sword['is_max'].observe(refresh_ui, names = 'value')

ui_layout = widgets.VBox([
    sword_num_is,
    base_container,
    output_area
])

display(ui_layout)
refresh_ui()